## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [22]:
# Separate categorical columns
categorical_columns = model_data.select_dtypes(exclude=[np.number]).columns

# Encode categorical variables
le = LabelEncoder()
for col in categorical_columns:
    model_data[col] = le.fit_transform(model_data[col])
    
# Define target and features   
y = model_data['Angaus']
X = model_data.drop('Angaus', axis=1)

# Split data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [23]:
# Define XGBoost model
model = xgb.XGBClassifier(n_estimators = 1000, learning_rate = 0.1,eval_metric="logloss", random_state = 808)

model.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds = 50)

best_trees = model.best_iteration
print(f"Best number of trees: {best_trees}")

[0]	validation_0-logloss:0.51354
[1]	validation_0-logloss:0.48803
[2]	validation_0-logloss:0.46738
[3]	validation_0-logloss:0.45126
[4]	validation_0-logloss:0.43766
[5]	validation_0-logloss:0.42555
[6]	validation_0-logloss:0.41728
[7]	validation_0-logloss:0.40769
[8]	validation_0-logloss:0.40082
[9]	validation_0-logloss:0.39562
[10]	validation_0-logloss:0.39263
[11]	validation_0-logloss:0.38984
[12]	validation_0-logloss:0.38489
[13]	validation_0-logloss:0.38327
[14]	validation_0-logloss:0.37803
[15]	validation_0-logloss:0.37652
[16]	validation_0-logloss:0.37337
[17]	validation_0-logloss:0.37156
[18]	validation_0-logloss:0.36987
[19]	validation_0-logloss:0.36868
[20]	validation_0-logloss:0.36809
[21]	validation_0-logloss:0.36721
[22]	validation_0-logloss:0.36763
[23]	validation_0-logloss:0.36570
[24]	validation_0-logloss:0.36577
[25]	validation_0-logloss:0.36666
[26]	validation_0-logloss:0.36559
[27]	validation_0-logloss:0.36552
[28]	validation_0-logloss:0.36422
[29]	validation_0-loglos

/Users/jocardelle/opt/anaconda3/envs/eds232-ml/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[71]	validation_0-logloss:0.38185
[72]	validation_0-logloss:0.38214
[73]	validation_0-logloss:0.38155
[74]	validation_0-logloss:0.38143
[75]	validation_0-logloss:0.38350
[76]	validation_0-logloss:0.38347
[77]	validation_0-logloss:0.38398
[78]	validation_0-logloss:0.38436
[79]	validation_0-logloss:0.38608
Best number of trees: 29


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [24]:
param_dist = {
    'learning_rate': uniform(0.01, 0.3)
}

randomized_cv = RandomizedSearchCV(
    estimator = model,
    param_distributions=param_dist,
    n_iter=20,
    random_state = 808
)

randomized_cv.fit(X_train, y_train)
learning_rate = randomized_cv.best_params_['learning_rate']
print(f"Best learning rate: {learning_rate}")

Best learning rate: 0.2041123012782783


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [25]:
# model with best trees and learning rate
best_model = XGBClassifier(
    n_estimators = model.best_iteration,
    learning_rate = learning_rate,
    eval_metric = "logloss",
    random_state = 808
)

param_dist = {
    'max_depth' : randint(3, 11),
    'min_child_weight' : randint(1, 11),
    'gamma': uniform(0.05, 0.05)
}

tree_cv = RandomizedSearchCV(
    estimator = best_model,
    param_distributions=param_dist,
    random_state = 808
)

tree_cv.fit(X_train, y_train)

print(f"best max_depth: {tree_cv.best_params_['max_depth']}")
print(f"best min_child_weight: {tree_cv.best_params_['min_child_weight']}")
print(f"best gamma: {tree_cv.best_params_['gamma']}")

best max_depth: 6
best min_child_weight: 9
best gamma: 0.09077113838342292


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and 1
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [26]:
model_2 = XGBClassifier(
    n_estimators = model.best_iteration,
    learning_rate = learning_rate,
    max_depth = tree_cv.best_params_['max_depth'],
    min_child_weight = tree_cv.best_params_['min_child_weight'],
    gamma = tree_cv.best_params_['gamma'],
    random_state = 808,
    eval_metric = "logloss",
    
)

param_dist = {
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5)
}

stoch_cv = RandomizedSearchCV(
    estimator = model_2,
    param_distributions= param_dist,
    random_state = 808
)

stoch_cv.fit(X_train, y_train)

print(f"subsample:{stoch_cv.best_params_['subsample']}")
print(f"col_sample_bytree: {stoch_cv.best_params_['colsample_bytree']}")

subsample:0.923170072812445
col_sample_bytree: 0.9347901290928311


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [28]:
eval_data = eval_data.rename(columns={'Angaus_obs': 'Angaus'})
eval_data.head()


,Angaus,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,0,16.6,1.01,1.017,5.23,0.29,-1.40,1.980,10.0,1.00,0,electric,4.9
1,1,16.8,-0.51,1.002,2.24,0.00,0.27,0.460,0.7,0.00,0,electric,2.3
2,0,16.3,0.76,1.023,162.28,5.14,-0.60,0.806,21.4,0.66,0,electric,4.3
3,0,15.6,1.56,1.003,4.05,0.57,1.14,3.300,0.9,0.75,0,net,1.0
4,0,14.6,-0.20,1.023,127.03,1.72,-1.90,1.940,28.9,0.97,0,electric,NaN


In [32]:

categorical_columns_eval = eval_data.select_dtypes(exclude=[np.number]).columns

for col in categorical_columns_eval:
    eval_data[col] = le.fit_transform(eval_data[col])
    
X_eval = eval_data.drop('Angaus', axis=1)
y_eval = eval_data['Angaus']

final_model = XGBClassifier(
    n_estimators=model.best_iteration,
    learning_rate=learning_rate,
    max_depth=tree_cv.best_params_['max_depth'],
    min_child_weight=tree_cv.best_params_['min_child_weight'],
    gamma=tree_cv.best_params_['gamma'],
    subsample=stoch_cv.best_params_['subsample'],
    colsample_bytree=stoch_cv.best_params_['colsample_bytree'],
    eval_metric="logloss",
    random_state=808
)

final_model.fit(X, y)

y_pred = final_model.predict(X_eval)
y_pred_proba = final_model.predict_proba(X_eval)[:, 1]

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [35]:
auc = roc_auc_score(y_eval, y_pred_proba)
print(f"AUC score: {auc}")

feature_importances = final_model.feature_importances_

importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance':feature_importances
}).sort_values('Importance', ascending=False)
print(f"feature importances: {importance}")

AUC score: 0.8778031438015743
feature importances:        Feature  Importance
0      SegSumT    0.259100
10      Method    0.143255
4   DSMaxSlope    0.085484
6   USRainDays    0.081710
8     USNative    0.075593
5       USAvgT    0.069512
3       DSDist    0.066731
7      USSlope    0.064481
11      LocSed    0.059685
1     SegTSeas    0.047392
2   SegLowFlow    0.047057
9        DSDam    0.000000


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

My model performed slightly better on the evaluation data than theirs on CV. For feature importances, we both had the most important predictor as SegSumT. However, my next most important was Method and theirs was USNative. We both had the same least important feature as DSDam. For predictive performance, my model is slightly better than the Elith model. We could also compare models on efficiency and on overfitting, if there is a large gap between the training and evaluation performance.


